In [ ]:
df = pd.read_csv("../../../user_classification/user_clusters.csv")

# Chuyển đổi ngày tháng
df['start_date'] = pd.to_datetime(df['start_date'])
df['target_date'] = pd.to_datetime(df['target_date'])
df['duration_days'] = (df['target_date'] - df['start_date']).dt.days

In [ ]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../../"))

sys.path.append(project_root)
from spending_forecast.forecast import ARIMAPredictor

predictor = ARIMAPredictor()
cluster = user_cluster 
# Dự đoán cho cụm 0
result = predictor.predict_cluster_spending(cluster)

spending_predict = result['EDU'] + result['PLAY']
saving_predict = result['FFA'] + result['LTSS']

Đã tải mô hình cho hũ NEC trong cụm 0
Đã tải mô hình cho hũ FFA trong cụm 0
Đã tải mô hình cho hũ EDU trong cụm 0
Đã tải mô hình cho hũ LTSS trong cụm 0
Đã tải mô hình cho hũ PLAY trong cụm 0
Đã tải mô hình cho hũ NEC trong cụm 1
Đã tải mô hình cho hũ FFA trong cụm 1
Đã tải mô hình cho hũ EDU trong cụm 1
Đã tải mô hình cho hũ LTSS trong cụm 1
Đã tải mô hình cho hũ PLAY trong cụm 1
Đã tải mô hình cho hũ NEC trong cụm 2
Đã tải mô hình cho hũ FFA trong cụm 2
Đã tải mô hình cho hũ EDU trong cụm 2
Đã tải mô hình cho hũ LTSS trong cụm 2
Đã tải mô hình cho hũ PLAY trong cụm 2
Đã tải mô hình cho hũ NEC trong cụm 3
Đã tải mô hình cho hũ FFA trong cụm 3
Đã tải mô hình cho hũ EDU trong cụm 3
Đã tải mô hình cho hũ LTSS trong cụm 3
Đã tải mô hình cho hũ PLAY trong cụm 3
{'NEC': 8200000, 'FFA': 2400000, 'EDU': 1800000, 'LTSS': 1800000, 'PLAY': 1800000, 'total': 16000000}


c:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\ASUS\Desktop\vpbank_hackathon\models_venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index b

In [23]:
print("Current working directory:", os.getcwd())
print("Project root:", project_root)


Current working directory: c:\Users\ASUS\Desktop\vpbank_hackathon\Ai_assitant\goals_recommendation\Difficulty_Rating
Project root: c:\Users\ASUS\Desktop\vpbank_hackathon\Ai_assitant


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Đọc file CSV
df = pd.read_csv("../../../user_data_ver1/user_goals_2saving_1spending.csv")

# Chuyển đổi ngày tháng
df['start_date'] = pd.to_datetime(df['start_date'])
df['target_date'] = pd.to_datetime(df['target_date'])
df['duration_days'] = (df['target_date'] - df['start_date']).dt.days

# Hàm tính toán các đặc trưng của mục tiêu
def extract_features(goal):
    return {
        'goal_type': 1 if goal['goal_type'] == 'saving' else 0,
        'goal_priority': {'Low': 0, 'Medium': 1, 'High': 2}[goal['goal_priority']],
        'goal_horizon': 1 if goal['goal_horizon'] == 'long' else 0,
        'target_amount': goal['target_amount'],
        'duration_days': goal['duration_days'], 
        'monthly_target' : goal['target_amount'] / (goal['duration_days'] / 30 )
        'feasibility' : 
    }

# Tạo vector đặc trưng cho tất cả mục tiêu
df['features'] = df.apply(extract_features, axis=1)

In [13]:
df['features'][0]

{'goal_type': 1,
 'goal_priority': 0,
 'goal_horizon': 1,
 'target_amount': 2346690000,
 'duration_days': 1770,
 'monthly_target': 39774406.779661015}

In [15]:
def find_similar_goals(new_goal, df, top_k=5):
    # Tạo vector đặc trưng cho mục tiêu mới
    new_features = extract_features(new_goal)
    new_vector = np.array([
        new_features['goal_type'],
        new_features['goal_priority'],
        new_features['goal_horizon'],
        new_features['target_amount'],
        new_features['duration_days'],
        new_features['monthly_target']
    ]).reshape(1, -1)
    
    # Tạo vector đặc trưng cho các mục tiêu hiện có
    vectors = np.array([
        [
            f['goal_type'],
            f['goal_priority'],
            f['goal_horizon'],
            f['target_amount'],
            f['duration_days'],
            f['monthly_target']
        ] for f in df['features']
    ])
    
    # Tính độ tương đồng cosine
    similarities = cosine_similarity(new_vector, vectors)[0]
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]
    
    # Lấy các mục tiêu tương tự
    similar_goals = df.iloc[top_k_indices]
    
    return similar_goals[['goal_id', 'user_id', 'completion_percent', 'target_amount', 'duration_days', 'associated_jar']]


In [17]:
def evaluate_new_goal(new_goal, df):
    similar_goals = find_similar_goals(new_goal, df)
    
    if similar_goals.empty:
        return {
            'feasibility': 'Không tìm thấy mục tiêu tương tự',
            'avg_completion': None,
            'suggestions': 'Kiểm tra lại thông tin mục tiêu hoặc thử với các tham số khác.'
        }
    
    avg_completion = similar_goals['completion_percent'].mean()
    feasibility = (
        'Khả thi' if avg_completion > 50 else
        'Khó đạt' if avg_completion > 20 else
        'Rất khó đạt'
    )
    
    # Rule-based suggestions
    suggestions = []
    if new_goal['goal_type'] == 'saving':
        avg_monthly_target = df[df['goal_type'] == 'saving']['features'].apply(lambda x: x['monthly_target']).mean()
        if new_goal['target_amount'] / (new_goal['duration_days'] / 30) > avg_monthly_target * 1.5:
            suggestions.append("Số tiền tiết kiệm hàng tháng có thể quá cao. Cân nhắc giảm target_amount hoặc kéo dài target_date.")
    elif new_goal['goal_type'] == 'spending':
        avg_spending_target = df[df['goal_type'] == 'spending']['target_amount'].mean()
        if new_goal['target_amount'] > avg_spending_target * 1.5:
            suggestions.append("Mục tiêu chi tiêu vượt mức trung bình. Cân nhắc giảm target_amount.")

    return {
        'feasibility': feasibility,
        'avg_completion': round(avg_completion, 2),
        'similar_goals': similar_goals.to_dict('records'),
        'suggestions': suggestions
    }

In [4]:
# Ví dụ mục tiêu mới
new_goal = {
    'user_id': 6,
    'goal_type': 'saving',
    'goal_priority': 'Medium',
    'goal_horizon': 'long',
    'target_amount': 3000000000,
    'start_date': pd.to_datetime('2025-07-01'),
    'target_date': pd.to_datetime('2028-07-01'),
    'associated_jar': 'EMERGENCY',
    'duration_days': (pd.to_datetime('2028-07-01') - pd.to_datetime('2025-07-01')).days
}

# Đánh giá mục tiêu
result = evaluate_new_goal(new_goal, df)

In [18]:
# Ví dụ mục tiêu mới
new_goal = {
    'user_id': 6,
    'goal_type': 'saving',
    'goal_priority': 'Medium',
    'goal_horizon': 'long',
    'target_amount': 3000000000,
    'start_date': pd.to_datetime('2025-07-01'),
    'target_date': pd.to_datetime('2028-07-01'),
    'associated_jar': 'EMERGENCY',
    'duration_days': (pd.to_datetime('2028-07-01') - pd.to_datetime('2025-07-01')).days
}

# Đánh giá mục tiêu
result = evaluate_new_goal(new_goal, df)
print("Đánh giá mục tiêu mới:")
print(f"Tính khả thi: {result['feasibility']}")
print(f"Tỷ lệ hoàn thành trung bình của các mục tiêu tương tự: {result['avg_completion']}%")
print("Các mục tiêu tương tự:")
for goal in result['similar_goals']:
    print(f"- Goal ID: {goal['goal_id']}, User ID: {goal['user_id']}, Completion: {goal['completion_percent']}%, Jar: {goal['associated_jar']}")
print("Gợi ý:")
for suggestion in result['suggestions']:
    print(f"- {suggestion}")

Đánh giá mục tiêu mới:
Tính khả thi: Khó đạt
Tỷ lệ hoàn thành trung bình của các mục tiêu tương tự: 25.27%
Các mục tiêu tương tự:
- Goal ID: 755, User ID: 279, Completion: 44.38%, Jar: LTSS
- Goal ID: 1047, User ID: 382, Completion: 4.18%, Jar: FFA
- Goal ID: 2096, User ID: 766, Completion: 22.71%, Jar: EMERGENCY
- Goal ID: 1769, User ID: 648, Completion: 17.36%, Jar: LTSS
- Goal ID: 194, User ID: 73, Completion: 37.72%, Jar: FFA
Gợi ý:
